In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Causal_Inference_Model/

/content/drive/MyDrive/Causal_Inference_Model


In [5]:
import sys, os

In [6]:
sys.path.append(os.path.abspath('scripts/'))

In [8]:
!pip install causalnex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 5.2 MB/s 
     |████████████████████████████████| 27.4 MB 78.6 MB/s 
     |████████████████████████████████| 1.9 MB 56.0 MB/s 
     |████████████████████████████████| 76 kB 4.1 MB/s 
     |████████████████████████████████| 22.3 MB 41.8 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 115 kB 64.9 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68719 sha256=055447acc1628368407a43a977a023d8e1932af155f3151f99a7fc52b1c842f8
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: scikit-learn
    Found existing in

Import Libraries

In [11]:
import warnings
import pandas as pd
from causalnex.structure import StructureModel
from causal_pipeline import EDAPipeline
from causal_helper import CausalHelper
warnings.filterwarnings("ignore")

In [13]:
preprocessed_trip_df = pd.read_csv("data/processed_trip.csv")
preprocessed_trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534369 entries, 0 to 534368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        534369 non-null  int64  
 1   Trip ID           534369 non-null  int64  
 2   Trip Origin       534369 non-null  object 
 3   Trip Destination  534369 non-null  object 
 4   Trip Start Time   534369 non-null  object 
 5   Trip End Time     534368 non-null  object 
 6   is_weekend        534369 non-null  int64  
 7   is_holiday        534369 non-null  int64  
 8   distance          534369 non-null  float64
 9   speed             534369 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 40.8+ MB


In [14]:
#Load the second data
df_driver = pd.read_csv("data/driver_locations_during_request.csv")
df_driver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


#### Merge the two Datasets

In [15]:
preprocessed_trip_df['driver_distance'] = preprocessed_trip_df['distance'].apply(lambda x: -1)
preprocessed_trip_df['driver_lat'] = preprocessed_trip_df['distance'].apply(lambda x: 0.0)
preprocessed_trip_df['driver_lng'] = preprocessed_trip_df['distance'].apply(lambda x: 0.0)

In [16]:
causalhelper = CausalHelper()

In [17]:
driver_lat,driver_lng = causalhelper.combine_get_driver_locations(preprocessed_trip_df, df_driver)

In [18]:
preprocessed_trip_df['driver_lat'] = driver_lat
preprocessed_trip_df['driver_lng'] = driver_lng

In [20]:
driver_lat_coordinates = preprocessed_trip_df['driver_lat'].tolist()
driver_lng_coordinates = preprocessed_trip_df['driver_lng'].tolist()
ending_coordinates = []
for i in range(len(driver_lat_coordinates)):
    ending_coordinates.append(f'{driver_lat_coordinates[i]},{driver_lng_coordinates[i]}')

In [21]:
start_coordinates = preprocessed_trip_df['Trip Origin'].tolist()

#### Calculate distance based on coordinates of driver

In [24]:
causal_pipeline = EDAPipeline()

In [25]:
preprocessed_trip_df['driver_distance'] = causal_pipeline.calculate_distances(start_coordinates, ending_coordinates)

Drop all the trips with -1 driver distance which means those orders were not in the 2nd dataset

In [26]:
preprocessed_trip_df_with_drivers = preprocessed_trip_df[preprocessed_trip_df['driver_distance']!=-1]
preprocessed_trip_df_with_drivers.shape

(26436, 13)

#### Compute unfulfilled orders

In [27]:
preprocessed_trip_df_with_drivers['is_fulfilled'] = preprocessed_trip_df_with_drivers['driver_distance'].apply(
    lambda x: 1 if x !=-2 else 0)

In [28]:
preprocessed_trip_df_with_drivers[preprocessed_trip_df_with_drivers['is_fulfilled']==0].shape

(609, 14)

#### Drop Unwanted columns

In [29]:
preprocessed_trip_df_with_drivers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26436 entries, 4 to 53812
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        26436 non-null  int64  
 1   Trip ID           26436 non-null  int64  
 2   Trip Origin       26436 non-null  object 
 3   Trip Destination  26436 non-null  object 
 4   Trip Start Time   26436 non-null  object 
 5   Trip End Time     26436 non-null  object 
 6   is_weekend        26436 non-null  int64  
 7   is_holiday        26436 non-null  int64  
 8   distance          26436 non-null  float64
 9   speed             26436 non-null  float64
 10  driver_distance   26436 non-null  float64
 11  driver_lat        26436 non-null  float64
 12  driver_lng        26436 non-null  float64
 13  is_fulfilled      26436 non-null  int64  
dtypes: float64(5), int64(5), object(4)
memory usage: 3.0+ MB


#### Store data

In [30]:
preprocessed_trip_df_with_drivers.to_csv("data/preprocessed_drivers.csv")

In [31]:
cols = ['Trip ID','Unnamed: 0', 'Trip Origin', 'Trip Destination', 'Trip Start Time', 'Trip End Time', 'driver_lat', 'driver_lng']
preprocessed_trip_df_with_drivers = preprocessed_trip_df_with_drivers.drop(columns=cols)

In [32]:
preprocessed_trip_df_with_drivers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26436 entries, 4 to 53812
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   is_weekend       26436 non-null  int64  
 1   is_holiday       26436 non-null  int64  
 2   distance         26436 non-null  float64
 3   speed            26436 non-null  float64
 4   driver_distance  26436 non-null  float64
 5   is_fulfilled     26436 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 1.4 MB


### Create Structural Model

In [33]:
smodel = StructureModel()

Create Edges
Transform and encode

In [34]:
import numpy as np

struct_data = preprocessed_trip_df_with_drivers.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

[]


In [35]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

### Scale and normalize numerical data

In [36]:
from sklearn import preprocessing

x = struct_data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

#### Apply the NOTEARS algorithm to learn the structure.

In [37]:
from causalnex.structure.notears import from_pandas

sm = from_pandas(df)

#### Visualize

In [42]:
!pip install graphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
!pip install pygraphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 118 kB 5.1 MB/s 
  ERROR: Failed building wheel for pygraphviz
  Running setup.py clean for pygraphviz
Failed to build pygraphviz
    Running setup.py install for pygraphviz ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-8lr4vfqn/pygraphviz_9496624d461143e694d82a3a1bc6bfab/setup.py'"'"'; __file__='"'"'/tmp/pip-install-8lr4vfqn/pygraphviz_9496624d461143e694d82a3a1bc6bfab/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-78d9erid/install-record.txt --single-version-externally-managed --comp

In [40]:
from IPython.display import Image
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE


In [41]:
viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
    
)
Image(viz.draw(format='png'))

Warning: ignored